In [1]:
!pip install tensorflow numpy matplotlib scikit-learn pillow pandas

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [27]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [28]:
train_dir = 'images/training'
test_dir = 'images/testing'

In [29]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [30]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [31]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 2367 images belonging to 3 classes.


In [32]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 591 images belonging to 3 classes.


In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [34]:
def create_baseline_model(input_shape=(224, 224, 3)):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

baseline_model = create_baseline_model()
baseline_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 54, 54, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 26, 26, 128)      

In [35]:
history_baseline = baseline_model.fit(train_generator, validation_data=test_generator, epochs=3)

Epoch 1/3
74/74 [==============================] - 280s 4s/step - loss: -14898040.0000 - accuracy: 0.4795 - val_loss: -116103712.0000 - val_accuracy: 0.4873
Epoch 2/3
74/74 [==============================] - 280s 4s/step - loss: -2652317440.0000 - accuracy: 0.4808 - val_loss: -10312916992.0000 - val_accuracy: 0.4873
Epoch 3/3
74/74 [==============================] - 304s 4s/step - loss: -50339033088.0000 - accuracy: 0.4808 - val_loss: -133815517184.0000 - val_accuracy: 0.4873


In [37]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [38]:
def create_resnet50_model(input_shape=(224, 224, 3)):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

resnet50_model = create_resnet50_model()
resnet50_model.summary()

94765736/94765736 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                         

In [39]:
history_resnet50 = resnet50_model.fit(train_generator, validation_data=test_generator, epochs=3)

Epoch 1/3
74/74 [==============================] - 470s 6s/step - loss: -32.8188 - accuracy: 0.4778 - val_loss: -78.0321 - val_accuracy: 0.4873
Epoch 2/3
74/74 [==============================] - 473s 6s/step - loss: -159.7166 - accuracy: 0.4808 - val_loss: -257.5944 - val_accuracy: 0.4873
Epoch 3/3
74/74 [==============================] - 470s 6s/step - loss: -400.7887 - accuracy: 0.4808 - val_loss: -557.3835 - val_accuracy: 0.4873


In [44]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Fine-tuning ResNet-50 model
def fine_tune_resnet50_model(model, unfreeze_layers=10):
    # Unfreeze the last `unfreeze_layers` layers of the base model
    for layer in model.layers[-unfreeze_layers:]:
        layer.trainable = True

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

fine_tuned_resnet50_model = fine_tune_resnet50_model(resnet50_model)

In [45]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [46]:
history_fine_tuned_resnet50 = fine_tuned_resnet50_model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=5,
    callbacks=[reduce_lr, early_stopping]
)

Epoch 1/5
74/74 [==============================] - 472s 6s/step - loss: -1473.1949 - accuracy: 0.4808 - val_loss: -1346.3750 - val_accuracy: 0.4873 - lr: 0.0010
Epoch 2/5
74/74 [==============================] - 473s 6s/step - loss: -2872.3188 - accuracy: 0.4808 - val_loss: -1919.8953 - val_accuracy: 0.4873 - lr: 0.0010
Epoch 3/5
74/74 [==============================] - 523s 7s/step - loss: -5120.5146 - accuracy: 0.4808 - val_loss: -4106.7607 - val_accuracy: 0.4873 - lr: 0.0010
Epoch 4/5
74/74 [==============================] - 474s 6s/step - loss: -8677.0537 - accuracy: 0.4808 - val_loss: -8183.1777 - val_accuracy: 0.4873 - lr: 0.0010
Epoch 5/5
74/74 [==============================] - 495s 7s/step - loss: -14018.0137 - accuracy: 0.4808 - val_loss: -17359.7539 - val_accuracy: 0.4873 - lr: 0.0010


In [47]:
from tensorflow.keras.applications import EfficientNetB0

In [48]:
def create_efficientnet_model(input_shape=(224, 224, 3)):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

efficientnet_model = create_efficientnet_model()
efficientnet_model.summary()

16705208/16705208 [==============================] - 0s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_2[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)   

In [49]:
history_efficientnet = efficientnet_model.fit(train_generator, validation_data=test_generator, epochs=3)

Epoch 1/3
74/74 [==============================] - 277s 4s/step - loss: -28.9342 - accuracy: 0.4782 - val_loss: -74.2481 - val_accuracy: 0.4873
Epoch 2/3
74/74 [==============================] - 260s 4s/step - loss: -156.1465 - accuracy: 0.4808 - val_loss: -265.5513 - val_accuracy: 0.4873
Epoch 3/3
74/74 [==============================] - 303s 4s/step - loss: -411.2669 - accuracy: 0.4808 - val_loss: -588.5508 - val_accuracy: 0.4873


In [53]:
from sklearn.metrics import f1_score

In [54]:
def get_predictions(model, data_generator):
    predictions = model.predict(data_generator)
    return (predictions > 0.5).astype(int)

In [55]:
baseline_preds = get_predictions(baseline_model, test_generator)
resnet50_preds = get_predictions(fine_tuned_resnet50_model, test_generator)
efficientnet_preds = get_predictions(efficientnet_model, test_generator)

19/19 [==============================] - 51s 3s/step


In [57]:
import numpy as np
ensemble_preds = np.round((baseline_preds + resnet50_preds + efficientnet_preds) / 3)

In [58]:
test_labels = test_generator.classes
f1 = f1_score(test_labels, ensemble_preds)
print(f'Ensemble Model F1 Score: {f1}')

Ensemble Model F1 Score: 0.6552901023890785
